In [1]:
import torch
import torch.nn as nn

In [13]:
#downsample = 차원을 줄이기위한 도구, stride가 2인 kernel의 convolutinon layer 사용
#stride는 kernal의 도약 범위 2면 2칸씩 진행 4이면 4칸씩 뛰어넘음
class block(nn.Module):
  def __init__(self,in_channel,out_channel,identity_downsample=None,stride=1):
    super(block,self).__init__()
    self.expansion = 4
    self.conv1 = nn.Conv2d(in_channel,out_channel,kernel_size = 1,stride=1,padding=0)
    self.bn1 = nn.BatchNorm2d(out_channel)
    self.conv2 = nn.Conv2d(out_channel,out_channel,kernel_size=3,stride =stride,padding=1)
    self.bn2 = nn.BatchNorm2d(out_channel)
    self.conv3 = nn.Conv2d(out_channel,out_channel*self.expansion,kernel_size=1,stride=1,padding=0)
    self.bn3 = nn.BatchNorm2d(out_channel*self.expansion)
    self.relu = nn.ReLU()
    self.identity_downsample =identity_downsample
  def forward_pass(self,x):
    identity = x
    x = self.conv1(x)
    x= self.bn1(x)
    x= self.relu(x)
    x= self.conv2(x)
    x= self.bn2(x)      #hidden batchnorm activation 의 3겹
    x= self.relu(x)
    x = self.conv3(x)
    x = self.bn3(x)
    x= self.relu(x)

    ##downsample 된 identity로 변형
    #잔차연결의 일부 x와 downsample된 identity의 차원을 맞추는 용도
    if self.identity_downsample is not None:

      identity = self.identity_downsample(identity)

    x+=identity # identity = x, 즉 conv bn relu 전의 input output을 내놓을때 input을 더 더하고 activation한뒤 return <-잔차연결
    x=self.relu(x)
    return x



class Resnet(nn.Module): #[3,4,6,3]
  def __init__(self, block, layers,image_channel, num_classes):
    super(Resnet,self).__init__()
    self.in_channel = 64
    self.conv1 = nn.Conv2d(image_channel, 64,kernel_size=7,stride = 2,padding=3)
    self.bn1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU()
    self.maxpool = nn.MaxPool2d(kernel_size = 3,stride =2,padding=1)

    #ResNet layer
    self.layer1 = self.build_layer(block,layers[0],out_channel=64,stride=1)
    self.layer2 = self.build_layer(block,layers[1],out_channel=128,stride=2)
    self.layer3 = self.build_layer(block,layers[2],out_channel=256,stride=2)
    self.layer4 = self.build_layer(block,layers[3],out_channel=512,stride=2)

    self.averagepool = nn.AdaptiveAvgPool2d((1,1))
    self.fullyconnected = nn.Linear(2048,num_classes)

  def forward_pass(self,x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.averagepool(x) #(batch_size, channels, 1, 1)
    x = x.reshape(x.shape[0],-1) #(batch_size, -1)
    x = self.fullyconnected(x)   #(2048,num_classes)
    return x


  def build_layer(self,block,num_residual_blocks,out_channel,stride):
    identity_downsample = None
    layers = []



      #nn.Sequential 은 여러 layers를 묶기 위한것
      #걍 model =nn.Sequential()하고 안에다nn.ReLU,nn.Conv2d 이런거 다 넣으면됨
      #어쨋든 stride가 2거나 in_channel/out_channel 4아닐때 downsample해서
      #stride1로 만들고 in,out 비를 4로 downsample함

    if stride !=1 or self.in_channel != out_channel*4:
      identity_downsample = nn.Sequential(nn.Conv2d(self.in_channel,out_channel*4,kernel_size = 1,
                                                    stride = stride),
                                          nn.BatchNorm2d(out_channel*4))



      #class block 사용 각각의 layer block포함 con2d,bn,relu *3
    layers.append(block(self.in_channel,out_channel,identity_downsample,stride))
      #하나 추가하고 update 왜냐 inchannel dim이 layer통과후 outchannel dim이 됬으므로
      #다음 layer inchannel dim = outchannel dim 4배다
    self.in_channel = out_channel*4

    for i in range(num_residual_blocks-1):
      layers.append(block(self.in_channel,out_channel)) #처음 256으로 증가한후 output까지 지속됨 차원

    return nn.Sequential(*layers)
    # 괄호안 * 기호 = unpacking e.g input_output =[2,3]  nn.Conv2d(*input_output) -> nn.conv2d(2,3)



In [11]:
#residual connection
#기본 layer , output = weight*input +bias
#but residual conenction, output = weight*input + input + bias
#transformer architecture에서도 사용됨
#autoencoder의 batchnorm layer output이 decoder의 multi head attention에 input에 더해짐
#Output=LayerNorm(Input+Sublayer(Input))


#resnet parameter (block,layers,image_channel,num_classes)
#layer만 조금씩 조정

def Resnet50(img_channel = 3,num_classes= 1000):
  return Resnet(block, [3,4,6,3],img_channel,num_classes)


def Resnet50(img_channel = 3,num_classes= 1000):
  return Resnet(block, [3,4,23,3],img_channel,num_classes)

def Resnet50(img_channel = 3,num_classes= 1000):
  return Resnet(block, [3,8,36,3],img_channel,num_classes)